[Reference](https://medium.com/swlh/python-learning-note-streamlit-d1be21a86f9f)

In [2]:
!streamlit hello

/bin/bash: streamlit: command not found


In [4]:
import streamlit as st
import numpy as np
import pandas as pd
import time
import serial
particle = serial.Serial('COM5', 9600, timeout=.1)

first_lock_1 = False
first_lock_2 = False
data_flag_1 = False
data_flag_2 = False

max_temperature = 0
min_temperature = 100
max_humidity = 0
min_humidity = 100

temp_array = np.array([])
humd_array = np.array([])

st.title('Environment Sensor monitor')
status_text_1 = st.empty()
status_text_2 = st.empty()
status_text_3 = st.empty()
status_text_4 = st.empty()
status_text_5 = st.empty()
status_text_6 = st.empty()

button_1 = st.button('Stop')

while button_1 != True:
    data = particle.readline()[:-2] #the last bit gets rid of the new-line chars

    if data:
        #Convert bytes to str data type
        data_str = data.decode('utf-8')
        data_str = data_str.split(" ")

        if  data_str[0] == 'Temperature:':

            #Convert str to float data type
            temp = float(data_str[1])
            print("Current temperature is: ")
            print(temp)
            data_flag_1 = True

            if temp >  max_temperature:
                max_temperature = temp
            if temp <  min_temperature:
                min_temperature = temp

        elif data_str[0] == 'Humidity:':

            #Convert str to float data type
            humd = float(data_str[1])
            print("Current Humidity is: ")
            print(humd)
            data_flag_2 = True

            if humd >  max_humidity:
                max_humidity = humd
            if humd <  min_humidity:
                min_humidity = humd

    #Create a chart first
    if data_flag_1==True and data_flag_2==True :
        if first_lock_1==False and first_lock_2==False:
            first_lock_2 = True
            first_lock_1 = True
            max_temperature = temp
            min_temperature = temp
            max_humidity = humd
            min_humidity = humd
            total_data = np.array([[temp,humd]])
            chart_data = pd.DataFrame(total_data,
                                      columns=['Temperature', 'Humidity'])
            chart = st.line_chart(chart_data)
            print("Initialize complete ... ")
        else:
            data_flag_1 = False
            data_flag_2 = False
            total_data = np.array([[temp,humd]])
            chart_data = pd.DataFrame(total_data,
                                      columns=['Temperature', 'Humidity'])
            chart.add_rows(chart_data)

            status_text_1.text('The latest temperature is: %3.2f' % temp)
            status_text_2.text('The highest temperature is: %3.2f' % max_temperature)
            status_text_3.text('The lowest temperature is: %3.2f' % min_temperature)

            status_text_4.text('The latest humidity is: %3.2f' % humd)
            status_text_5.text('The highest temperature is: %3.2f' % max_humidity)
            status_text_6.text('The lowest temperature is: %3.2f' % min_humidity)